In [ ]:
''' 2. Create a dual-axis chart comparing the average installs and revenue for free vs. paid apps within the top 3 app categories. 
Apply filters to exclude apps with fewer than 10,000 installs and revenue below $10,000 and android version should be more than 4.0 as well as size should be more than 15M and content rating should be Everyone and app name should not have more than 30 characters including space and special character .
This graph should work only between 1 PM IST to 2 PM IST apart from that time we should not show this graph in dashboard itself.'''

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
apps_df = pd.read_csv("E:/Data Analyst Intern Tasks/PlayStore Data.csv")
reviews_df = pd.read_csv("E:/Data Analyst Intern Tasks/User reviews.csv")

In [ ]:
apps_df = apps_df.loc[:, ~apps_df.columns.str.contains('^Unnamed')]
apps_df

In [ ]:
reviews_df.head()

In [ ]:
# Data Cleaning

apps_df = apps_df.dropna(subset=['Rating'])
for column in apps_df.columns:
    apps_df[column].fillna(apps_df[column].mode()[0])
apps_df.drop_duplicates   
apps_df = apps_df[apps_df['Rating']<=5]
reviews_df.dropna(subset=['Translated_Review'],inplace=True)

In [ ]:
#Convert the Installs columns to numeric by removing commas and +
apps_df['Installs']=apps_df['Installs'].str.replace(',','').str.replace('+','').astype(int)

# Convert Price column to numeric after removing $
apps_df['Price']=apps_df['Price'].str.replace('$','').astype(float)

In [ ]:
# For missing Android versions, we can use a placeholder like "Not specified"
apps_df['Android Ver'] = apps_df['Android Ver'].fillna('Not specified')

# For missing Current Ver, we can either drop these rows or fill with a placeholder
apps_df['Current Ver'] = apps_df['Current Ver'].fillna('Not specified')

In [ ]:
apps_df.dtypes

In [ ]:
apps_df.isnull().sum()

In [ ]:
reviews_df.dtypes

In [ ]:
reviews_df.isnull().sum()

In [ ]:
apps_df.columns

In [ ]:
reviews_df.columns

In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.ticker import FuncFormatter
import datetime
from matplotlib.patches import Patch

In [ ]:
class AppDashboard(tk.Tk):
    def __init__(self, apps_df):
        super().__init__()
        self.title("Google Play Store Analysis Dashboard")
        self.geometry("1200x800")
        self.configure(bg="lightgray")
        self.apps_df = apps_df

        main_frame = tk.Frame(self)
        main_frame.pack(fill=tk.BOTH, expand=True)

        # Canvas and scrollbars
        canvas = tk.Canvas(main_frame, bg='lightgray')
        v_scrollbar = ttk.Scrollbar(main_frame, orient="vertical", command=canvas.yview)
        h_scrollbar = ttk.Scrollbar(main_frame, orient="horizontal", command=canvas.xview)
        v_scrollbar.pack(side="right", fill="y")
        h_scrollbar.pack(side="bottom", fill="x")
        canvas.pack(side="left", fill="both", expand=True)

        # Scrollable frame
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.config(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)

        # Grid configuration
        for i in range(6):
            scrollable_frame.columnconfigure(i, weight=1)

        # Add visualizations
        self.create_category_analysis(scrollable_frame, 0, 0)
        self.create_type_analysis(scrollable_frame, 0, 1)
        self.create_rating_analysis(scrollable_frame, 0, 2)
        self.create_dual_axis_chart(scrollable_frame, 1, 0)
        self.create_installation_analysis(scrollable_frame, 2, 0)
        self.create_revenue_analysis(scrollable_frame, 2, 1)

    def create_dual_axis_chart(self, parent, row, column):
        frame = ttk.Frame(parent, padding="5")
        frame.grid(row=row, column=column, sticky="nsew", pady=5, columnspan=3)
        frame.columnconfigure(0, weight=1)

        # Check time window (1 PM to 2 PM IST)
        current_time = datetime.datetime.now().time()
        if not (datetime.time(13, 0) <= current_time <= datetime.time(14, 0)):
            ttk.Label(frame, text="This chart only appears between 1 PM - 2 PM IST", 
                     foreground="gray").pack(expand=True)
            return

        try:
            filtered_df = self.apps_df.copy()
            
            # 1. Filter installs >= 10,000
            filtered_df = filtered_df[filtered_df['Installs'] >= 10000]
            
            # 2. Calculate and filter revenue >= $10,000
            filtered_df['Revenue'] = filtered_df['Price'] * filtered_df['Installs']
            filtered_df = filtered_df[filtered_df['Revenue'] >= 10000]
            
            # 3. Filter Android version > 4.0
            filtered_df = filtered_df[filtered_df['Android Ver'].apply(
                lambda x: float(str(x).split(' ')[0]) > 4.0 if pd.notna(x) else False)]
            
            # 4. Filter size > 15M
            filtered_df = filtered_df[filtered_df['Size'] > 15]
            
            # 5. Filter content rating is 'Everyone'
            filtered_df = filtered_df[filtered_df['Content Rating'] == 'Everyone']
            
            # 6. Filter app name length <= 30 characters
            filtered_df = filtered_df[filtered_df['App'].str.len() <= 30]
            
            # Check if data exists after filtering
            if filtered_df.empty:
                raise ValueError("No apps meet all filtering criteria")
            
            # Get top 3 categories and group data
            top_categories = filtered_df['Category'].value_counts().nlargest(3).index
            filtered_df = filtered_df[filtered_df['Category'].isin(top_categories)]
            
            grouped = filtered_df.groupby(['Type', 'Category']).agg({'Installs': 'mean','Revenue': 'mean'}).reset_index()
            
            # Pivot for plotting
            pivot_installs = grouped.pivot(index='Category', columns='Type', values='Installs')
            pivot_revenue = grouped.pivot(index='Category', columns='Type', values='Revenue')
            
            # Create figure ax1 for left axis and ax2 for right axis
            fig, ax1 = plt.subplots(figsize=(12, 6))
            
            # Plot installs (left axis)
            pivot_installs.plot(kind='bar', ax=ax1, color=['#1f77b4', '#ff7f0e'], 
                              width=0.4, position=1, alpha=0.8, edgecolor='black')
            ax1.set_ylabel('Average Installs', color='#1f77b4')
            ax1.tick_params(axis='y', labelcolor='#1f77b4')
            
            # Plot revenue (right axis)
            ax2 = ax1.twinx()
            pivot_revenue.plot(kind='bar', ax=ax2, color=['#2ca02c', '#d62728'], 
                             width=0.4, position=0, alpha=0.6, edgecolor='black')
            ax2.set_ylabel('Average Revenue ($)', color='#2ca02c')
            ax2.tick_params(axis='y', labelcolor='#2ca02c')
            
            # Formatting
            ax1.set_title('Free vs Paid Apps: Installs & Revenue Comparison (Top 3 Categories)')
            ax1.set_xlabel('Category')
            
            # Custom legend
            legend_elements = [
                Patch(facecolor='#1f77b4', label='Free Apps - Installs'),
                Patch(facecolor='#ff7f0e', label='Paid Apps - Installs'),
                Patch(facecolor='#2ca02c', label='Free Apps - Revenue'),
                Patch(facecolor='#d62728', label='Paid Apps - Revenue')]
            ax1.legend(handles=legend_elements, loc='upper right')
            
            # Add to Tkinter(Python GUI)
            canvas = FigureCanvasTkAgg(fig, master=frame)
            canvas.draw()
            canvas.get_tkinter_widget().pack(expand=True, fill="both")
            
        except Exception as e:
            ttk.Label(frame, text=f"Error generating chart: {str(e)}", 
                     foreground="red").pack(expand=True)

    # Other visualization methods 
    def create_category_analysis(self, parent, row, column):
        pass  # Implementation would be similar to existing code
    
    def create_type_analysis(self, parent, row, column):
        pass 
    
    def create_rating_analysis(self, parent, row, column):
        pass  
    
    def create_installation_analysis(self, parent, row, column):
        pass 
    
    def create_revenue_analysis(self, parent, row, column):
        pass  

if __name__ == "__main__":
    apps_df = pd.read_csv("E:/Data Analyst Intern Tasks/PlayStore Data.csv")
    
    # Data cleaning (add your existing cleaning code here)
    # Data Cleaning
    apps_df = apps_df.dropna(subset=['Rating'])
    for column in apps_df.columns:
        apps_df[column].fillna(apps_df[column].mode()[0])
    apps_df.drop_duplicates   
    apps_df = apps_df[apps_df['Rating']<=5]
    reviews_df.dropna(subset=['Translated_Review'],inplace=True)

    #Convert the Installs columns to numeric by removing commas and +
    apps_df['Installs']=apps_df['Installs'].str.replace(',','').str.replace('+','').astype(int)

    # Convert Price column to numeric after removing $
    apps_df['Price']=apps_df['Price'].str.replace('$','').astype(float)

    # For missing Android versions, we can use a placeholder like "Not specified"
    apps_df['Android Ver'] = apps_df['Android Ver'].fillna('Not specified')

    # For missing Current Ver, we can either drop these rows or fill with a placeholder
    apps_df['Current Ver'] = apps_df['Current Ver'].fillna('Not specified')
    
    # Create and run dashboard
    dashboard = AppDashboard(apps_df)
    dashboard.mainloop()